In [1]:
import pandas as pd

In [2]:
big_table_train_df = pd.read_csv('datasets/big-table-train.csv')

In [3]:
big_table_train_df.head()

,Id,running_time,completed_time,route_distance_km,delta_time,node_start,node_finish,distance,speed,average_speed,lon_start,lat_start,lon_end,lat_end
0,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.74,862.0,10980417,4768348185,31.771489,30.0,29.537736,30.738275,46.468524,30.738097,46.468266
1,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.74,862.0,10980418,1570776947,3.673054,24.0,29.537736,NaN,NaN,NaN,NaN
2,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.74,862.0,10980430,4768348188,15.550612,27.0,29.537736,NaN,NaN,NaN,NaN
3,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.74,862.0,10980431,5125489557,50.034390,29.0,29.537736,30.739016,46.471232,30.738957,46.470784
4,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.74,862.0,10980432,5213300015,13.453126,30.0,29.537736,30.739248,46.472665,30.739226,46.472545


In [4]:
big_table_train_df['running_time'] = pd.to_datetime(big_table_train_df['running_time'])
big_table_train_df['completed_time'] = pd.to_datetime(big_table_train_df['completed_time'])

In [5]:
big_table_train_df

,Id,running_time,completed_time,route_distance_km,delta_time,node_start,node_finish,distance,speed,average_speed,lon_start,lat_start,lon_end,lat_end
0,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,10980417,4768348185,31.771489,30.0,29.537736,30.738275,46.468524,30.738097,46.468266
1,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,10980418,1570776947,3.673054,24.0,29.537736,NaN,NaN,NaN,NaN
2,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,10980430,4768348188,15.550612,27.0,29.537736,NaN,NaN,NaN,NaN
3,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,10980431,5125489557,50.034390,29.0,29.537736,30.739016,46.471232,30.738957,46.470784
4,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,10980432,5213300015,13.453126,30.0,29.537736,30.739248,46.472665,30.739226,46.472545
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401306,-4845695090750648713,2022-01-24 14:57:46,2022-01-24 15:11:25,2.323,819.0,8952395527,8952395526,9.279770,24.0,23.222222,NaN,NaN,NaN,NaN
401307,-4845695090750648713,2022-01-24 14:57:46,2022-01-24 15:11:25,2.323,819.0,8952395528,8952395527,90.006624,18.0,23.222222,NaN,NaN,NaN,NaN
401308,-4845695090750648713,2022-01-24 14:57:46,2022-01-24 15:11:25,2.323,819.0,8952395529,8952395528,59.390484,20.0,23.222222,NaN,NaN,NaN,NaN
401309,-4845695090750648713,2022-01-24 14:57:46,2022-01-24 15:11:25,2.323,819.0,8952395530,8952395529,7.099413,24.0,23.222222,NaN,NaN,NaN,NaN


In [6]:
big_table_train_df["hours"] = big_table_train_df['running_time'].dt.hour
big_table_train_df

,Id,running_time,completed_time,route_distance_km,delta_time,node_start,node_finish,distance,speed,average_speed,lon_start,lat_start,lon_end,lat_end,hours
0,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,10980417,4768348185,31.771489,30.0,29.537736,30.738275,46.468524,30.738097,46.468266,18
1,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,10980418,1570776947,3.673054,24.0,29.537736,NaN,NaN,NaN,NaN,18
2,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,10980430,4768348188,15.550612,27.0,29.537736,NaN,NaN,NaN,NaN,18
3,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,10980431,5125489557,50.034390,29.0,29.537736,30.739016,46.471232,30.738957,46.470784,18
4,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,10980432,5213300015,13.453126,30.0,29.537736,30.739248,46.472665,30.739226,46.472545,18
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401306,-4845695090750648713,2022-01-24 14:57:46,2022-01-24 15:11:25,2.323,819.0,8952395527,8952395526,9.279770,24.0,23.222222,NaN,NaN,NaN,NaN,14
401307,-4845695090750648713,2022-01-24 14:57:46,2022-01-24 15:11:25,2.323,819.0,8952395528,8952395527,90.006624,18.0,23.222222,NaN,NaN,NaN,NaN,14
401308,-4845695090750648713,2022-01-24 14:57:46,2022-01-24 15:11:25,2.323,819.0,8952395529,8952395528,59.390484,20.0,23.222222,NaN,NaN,NaN,NaN,14
401309,-4845695090750648713,2022-01-24 14:57:46,2022-01-24 15:11:25,2.323,819.0,8952395530,8952395529,7.099413,24.0,23.222222,NaN,NaN,NaN,NaN,14


In [7]:
big_table_train_df["is_work_hour"] = [1 if (i >= 8 and i <= 18) else 0 for i in big_table_train_df['hours']]
big_table_train_df['is_night_time'] = [1 if (i >= 19 and i <= 23) else 0 for i in big_table_train_df['hours']]
big_table_train_df['is_late_night_time'] = [1 if (i >= 0 and i <= 5) else 0 for i in big_table_train_df['hours']]
big_table_train_df['is_rush_hour'] = [1 if ((i >= 6 and i <= 9) or (i >= 17 and i <= 20)) else 0 for i in big_table_train_df['hours']]

In [8]:
big_table_train_df

,Id,running_time,completed_time,route_distance_km,delta_time,node_start,node_finish,distance,speed,average_speed,lon_start,lat_start,lon_end,lat_end,hours,is_work_hour,is_night_time,is_late_night_time,is_rush_hour
0,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,10980417,4768348185,31.771489,30.0,29.537736,30.738275,46.468524,30.738097,46.468266,18,1,0,0,1
1,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,10980418,1570776947,3.673054,24.0,29.537736,NaN,NaN,NaN,NaN,18,1,0,0,1
2,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,10980430,4768348188,15.550612,27.0,29.537736,NaN,NaN,NaN,NaN,18,1,0,0,1
3,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,10980431,5125489557,50.034390,29.0,29.537736,30.739016,46.471232,30.738957,46.470784,18,1,0,0,1
4,-4773019581999572651,2022-01-24 18:30:21,2022-01-24 18:44:43,3.740,862.0,10980432,5213300015,13.453126,30.0,29.537736,30.739248,46.472665,30.739226,46.472545,18,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
401306,-4845695090750648713,2022-01-24 14:57:46,2022-01-24 15:11:25,2.323,819.0,8952395527,8952395526,9.279770,24.0,23.222222,NaN,NaN,NaN,NaN,14,1,0,0,0
401307,-4845695090750648713,2022-01-24 14:57:46,2022-01-24 15:11:25,2.323,819.0,8952395528,8952395527,90.006624,18.0,23.222222,NaN,NaN,NaN,NaN,14,1,0,0,0
401308,-4845695090750648713,2022-01-24 14:57:46,2022-01-24 15:11:25,2.323,819.0,8952395529,8952395528,59.390484,20.0,23.222222,NaN,NaN,NaN,NaN,14,1,0,0,0
401309,-4845695090750648713,2022-01-24 14:57:46,2022-01-24 15:11:25,2.323,819.0,8952395530,8952395529,7.099413,24.0,23.222222,NaN,NaN,NaN,NaN,14,1,0,0,0
